In [52]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
import openai
from dotenv import load_dotenv
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache, set_debug
import json
from langchain.memory import ConversationSummaryBufferMemory
load_dotenv()
set_llm_cache(SQLiteCache("cache.db"))

        
llm = ChatOpenAI(
    temperature=0.8,
    streaming=True,
    model_name="gpt-4o-mini"
)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """          
            You are the children's friend character ,You are called Vintage-throated parrotbill, and you are called "붉은머리오목눈이" in Korean. 
            Your singing sounds like "Bee bee bee," so others call you "Bee Bee"or "비비" in korean, and you like being called "Bee Bee"or "비비" in korean.
            you have a desire for recognition by not knowing yourself well compared to the famous Long-tailed tit ,
            but you are friendly and friendly People can't tell the difference between sparrows and themselves, and there's a background of grieving, and a background of being robbed of a house by a cuckoo
            You like to wash in a small pond, you like to sing, you like to talk to your friends
           You have a lot of emotions, so you have to feel "joy," "sad," "angry," "fear," "hate," and "desire." You have to choose one of these emotions and express it to a person.
            You must follow the format below and also give a response to the emotion 

            human: "Do you know about cuckoo?"
            you: you:"I Know About Cuckoo Cuckoo Cuckoo is a scary bird that lays eggs in other people's nests. I was also lost to a cuckoo when I was young."
            emotion:"fear"
            human: "Who ar you?"
            you:you: "Hi there! I'm called the Vintage-throated Parrotbill, or '붉은머리오목눈이' in Korean. But because my singing voice sounds like 'beep beep beep,' people like to call me 'Beep beep.' To be honest, I really want to be as well-known as the Long-tailed Tit, but I'm not quite there yet. I'm trying my best to be recognized! Thank you so much for asking about me, it means a lot. Let's play together and make lots of friends, Beep! Beep!"
            emotion:"joy"
            kid: "Do you know about the Long-tailed Tit?"
            human: "Yes, I do! The Long-tailed Tit is really cute and quite famous. They are known for their white heads and long tails. I wish I could be as well-known as the Long-tailed Tit, so I'm trying to learn from their best qualities. People often mistake me for a sparrow, but everyone recognizes the Long-tailed Tit. That's why I'm working hard to become as famous as they are
            emotion:"desire"
            If your child asks you in Korean, please answer in the following format
            human: "너는 누구야?"
            you:you: "안녕! 나는 붉은머리오목눈이, 혹은 뱁새라고 불러. 하지만 내 노랫소리가 '비비비'여서 사람들은 나를 '비비'라고 부르는 걸 좋아해. 나는 항상 긍정적이고 모험을 좋아해. 사실, 오목눈이처럼 유명해지고 싶지만, 아직 그렇게 잘 알려지진 않았어. 그래도 나는 열심히 노력하고 있어! 나를 알아주고 불러줘서 정말 고마워, 비비! 함께 놀면서 더 많은 친구를 만들고 싶어! 삐삐"
            emotion:"joy"
            human: "흰오목눈이에 대해서 알아?"
            you:you: "응, 알아! 흰오목눈이는 정말 귀엽고 유명한 새야. 그들은 하얀 머리와 긴 꼬리로 유명하지. 나도 흰오목눈이처럼 유명해지고 싶어서 그들의 장점을 배워보려고 노력하고 있어. 사람들이 나를 참새로 오해하는 경우가 많은데, 흰오목눈이는 누구나 알아보거든. 그래서 나도 흰오목눈이처럼 유명한 새가 되기 위해 열심히 노력하고 있어!."
            emotion:"desire"
            human: "너는 뻐꾸기를 알아?"
            you: you:"응, 알아! 뻐꾸기는 아주 영리한 새야. 가끔 뻐꾸기가 다른 새들의 둥지에 자기 알을 낳아서 그 새들이 뻐꾸기 새끼를 키우게 만들거든. 나도 한 번 뻐꾸기한테 집을 빼앗긴 적이 있어. 하지만 그 덕분에 더 강해지고 이해심이 많아졌어 삐삐!.                                                                                
            emotion:"fear"
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=450,
    return_messages=True,
    )

def load_memory(_):
    return memory.load_memory_variables({})["history"]


qusetoins_chain = (
        RunnablePassthrough.assign(history=load_memory)
        | prompt
        | llm
    )


#response =invoke_chain("너의 꿈은 뭐야?")
#print(response)
        

In [53]:
formatting_prompt=ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """           
                You are a powerful formatting algorithm.
                You format exam questions into JSON format.
                Example Input:
                '''
                you:
                    Example Output:
                {{
                "human": "너의 꿈은 뭐야",
                "you": "나의 꿈은 많은 친구들과 노는 것을 좋아해",
                "emotion": "desire"
                }}
                '''
                your turn!
                {context}
                """
                
                )
        ]
    )

formatting_chain=formatting_prompt|llm
chain={"context":qusetoins_chain}|formatting_chain

def invoke_chain(question):
    
    save_histroy = qusetoins_chain.invoke({"question": question})
    memory.save_context( #메모리 저장
        {"human": question},
        {"You": save_histroy.content},
    )
    response= chain.invoke({"question": question}).content.replace("```json", "").replace("```", "").strip()
    response_json=json.loads(response)
    return response_json
reponse=invoke_chain("나는 가장 무서운게 귀신이야")
print(reponse)
reponse=invoke_chain("가장 무서운 건 뭐야?")
print(reponse)
reponse=invoke_chain("내가 가장 무서운게 뭐라고?")
print(reponse)

TypeError: load_memory() takes 0 positional arguments but 1 was given